In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import os

In [ ]:
!pip install pylabel

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 367.8/367.8 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 11.4 MB/s eta 0:00:00


In [ ]:
import os
import shutil
import xml.etree.ElementTree as ET
from tqdm import tqdm

def convert_to_yolo(size, box):
    dw = 1.0 / size[0]
    dh = 1.0 / size[1]
    x = (box[0] + box[1]) / 2.0
    y = (box[2] + box[3]) / 2.0
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = x * dw
    w = w * dw
    y = y * dh
    h = h * dh
    return (x, y, w, h)

dataset_path = "/content/drive/MyDrive/CA_2024/MaskedFace/train"
save_dataset_path = "/content/drive/MyDrive/CA_2024/YoloMasked"
os.makedirs(save_dataset_path, exist_ok=True)

train_path = os.path.join(save_dataset_path, "train")
os.makedirs(train_path, exist_ok=True)
train_images_path = os.path.join(train_path, "images")
os.makedirs(train_images_path, exist_ok=True)
train_labels_path = os.path.join(train_path, "labels")
os.makedirs(train_labels_path, exist_ok=True)

image_files = [f for f in os.listdir(dataset_path) if f.endswith(".png")]

class_names = ['with_mask', 'without_mask', 'mask_weared_incorrect']
class_indices = {class_name: i for i, class_name in enumerate(class_names)}

for image_file in tqdm(image_files):
    image_path = os.path.join(dataset_path, image_file)
    label_file = os.path.splitext(image_file)[0] + ".xml"
    label_path = os.path.join(dataset_path, label_file)

    if os.path.exists(image_path) and os.path.exists(label_path):
        shutil.copy(image_path, train_images_path)
        tree = ET.parse(label_path)
        root = tree.getroot()
        size = root.find("size")
        width = int(size.find("width").text)
        height = int(size.find("height").text)
        yolo_label_file = os.path.join(train_labels_path, os.path.splitext(image_file)[0] + ".txt")
        with open(yolo_label_file, "w") as f:
            for obj in root.iter("object"):
                class_name = obj.find("name").text
                if class_name not in class_indices:
                    continue  # Skip unknown classes
                class_idx = class_indices[class_name]
                bbox = obj.find("bndbox")
                xmin = float(bbox.find("xmin").text)
                xmax = float(bbox.find("xmax").text)
                ymin = float(bbox.find("ymin").text)
                ymax = float(bbox.find("ymax").text)
                x, y, w, h = convert_to_yolo((width, height), (xmin, xmax, ymin, ymax))
                f.write(f"{class_idx} {float(x)} {float(y)} {float(w)} {float(h)}\n")


In [ ]:
dataset_path = "/content/drive/MyDrive/CA_2024/MaskedFace/val"
save_dataset_path = "/content/drive/MyDrive/CA_2024/YoloMasked"
os.makedirs(save_dataset_path, exist_ok=True)

train_path = os.path.join(save_dataset_path, "val")
os.makedirs(train_path, exist_ok=True)
train_images_path = os.path.join(train_path, "images")
os.makedirs(train_images_path, exist_ok=True)
train_labels_path = os.path.join(train_path, "labels")
os.makedirs(train_labels_path, exist_ok=True)

image_files = [f for f in os.listdir(dataset_path) if f.endswith(".png")]

class_names = ['with_mask', 'without_mask', 'mask_weared_incorrect']
class_indices = {class_name: i for i, class_name in enumerate(class_names)}

for image_file in tqdm(image_files):
    image_path = os.path.join(dataset_path, image_file)
    label_file = os.path.splitext(image_file)[0] + ".xml"
    label_path = os.path.join(dataset_path, label_file)

    if os.path.exists(image_path) and os.path.exists(label_path):
        shutil.copy(image_path, train_images_path)
        tree = ET.parse(label_path)
        root = tree.getroot()
        size = root.find("size")
        width = int(size.find("width").text)
        height = int(size.find("height").text)
        yolo_label_file = os.path.join(train_labels_path, os.path.splitext(image_file)[0] + ".txt")
        with open(yolo_label_file, "w") as f:
            for obj in root.iter("object"):
                class_name = obj.find("name").text
                if class_name not in class_indices:
                    continue
                class_idx = class_indices[class_name]
                bbox = obj.find("bndbox")
                xmin = float(bbox.find("xmin").text)
                xmax = float(bbox.find("xmax").text)
                ymin = float(bbox.find("ymin").text)
                ymax = float(bbox.find("ymax").text)
                x, y, w, h = convert_to_yolo((width, height), (xmin, xmax, ymin, ymax))
                f.write(f"{class_idx} {float(x)} {float(y)} {float(w)} {float(h)}\n")

In [ ]:
import os
import cv2
from tqdm import tqdm
import matplotlib.pyplot as plt

images_path = "/content/drive/MyDrive/CA_2024/YoloMasked/train/images"
labels_path = "/content/drive/MyDrive/CA_2024/YoloMasked/train/labels"

image_files = [f for f in os.listdir(images_path) if f.endswith(".png")]


def draw_boxes(image, label_file):
    with open(label_file, "r") as f:
        for line in f:
            class_name, x, y, w, h = line.strip().split()
            x, y, w, h = float(x), float(y), float(w), float(h)
            x1 = int((x - w / 2) * image.shape[1])
            y1 = int((y - h / 2) * image.shape[0])
            x2 = int((x + w / 2) * image.shape[1])
            y2 = int((y + h / 2) * image.shape[0])
            cv2.rectangle(image, (x1, y1), (x2, y2), (255, 0, 0), 2)
            cv2.putText(image, class_name, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
i=0
for image_file in tqdm(image_files):
    image_path = os.path.join(images_path, image_file)
    label_file = os.path.join(labels_path, os.path.splitext(image_file)[0] + ".txt")

    # Read the image
    image = cv2.imread(image_path)
    if image is None:
        continue

    # Draw bounding boxes
    draw_boxes(image, label_file)

    # Display the image using plt
    plt.figure(figsize=(10, 10))
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.axis("off")
    plt.show()
    if i==3:
        break
    i=i+1


In [ ]:
# Install the ultralytics package from PyPI
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.5/719.5 kB 10.1 MB/s eta 0:00:00


In [ ]:
!wandb off

/bin/bash: line 1: wandb: command not found


In [ ]:
from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()
from ultralytics import YOLO

from IPython.display import display, Image


Ultralytics YOLOv8.1.24 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla V100-SXM2-16GB, 16151MiB)
Setup complete ✅ (8 CPUs, 51.0 GB RAM, 26.4/201.2 GB disk)


In [ ]:
!apt-get update
!apt-get install libgl1-mesa-glx -y

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [734 kB]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [1,894 kB]
Hit:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.com/

In [ ]:
os.chdir('/content/drive/MyDrive/CA_2024/YoloMasked')

In [ ]:
%%writefile data.yaml
train: "/content/drive/MyDrive/CA_2024/YoloMasked/train"
val: "/content/drive/MyDrive/CA_2024/YoloMasked/val"

nc: 3  # number of classes
names: ['with_mask', 'without_mask', 'mask_weared_incorrect']

Overwriting data.yaml


In [ ]:
!yolo mode=train model=yolov8l.pt iou=0.5 imgsz=640 data=data.yaml epochs=50 batch=8 name=yolov8n_custom augment=false optimizer=AdamW lr0=0.00005 lrf=0.01 crop_fraction=0 fliplr=0 scale=0 translate=0 hsv_h=0 hsv_s=0 hsv_v=0

Ultralytics YOLOv8.1.24 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
engine/trainer: task=detect, mode=train, model=yolov8l.pt, data=data.yaml, epochs=50, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=yolov8n_custom4, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.5, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True,